In [45]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
import pandas as pd

In [47]:
with pd.ExcelFile('Inputs.xlsx') as book:
    risk_ranking_weights = pd.read_excel(book, 'Risk Ranking Weights')
    non_union_rates = pd.read_excel(book, 'Non-Union Rates')
    union_rates = pd.read_excel(book, 'Union Rates')
    checkpoint_costs = pd.read_excel(book, 'Checkpoint Costs')
    df = pd.read_excel(book, 'merged_data_union')

In [48]:
# Fix formating
df['DateCreated'] = pd.to_datetime(df['DateCreated'], format = 'mixed', )
df['DateFixed'] = pd.to_datetime(df['DateFixed'], format = 'mixed')

In [49]:
df

,,DeficiencyID,DateCreated,DateFixed,CheckpointStatus,RiskRanking,Checklist ID,Checklist Category,Checklist Code,Checklist Item,...,Notes,ResponsibleParty,Project,Location,Union / Non-Union,Time taken to complete task (in mins),Minute Rate (based on union/non-union),Cost to re-do (based on hourly wage and multiplier),Material Cost,Total cost to re-do task
0,3096177,116797739,2024-06-22,2024-06-07,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,Quality Auditing Compliance-LG,JA,Suncor - Firebag (48),Western (48-1),Union,NaN,NaN,NaN,NaN,NaN
1,3229935,122354816,2024-10-26,2024-10-26,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,"Auditor to select|deficiency, Reason Code and ...",JA,Suncor - Mount Logan (49),Western (49-1),Union,NaN,NaN,NaN,NaN,NaN
2,3227124,122232426,2024-10-20,2024-10-20,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,| Fridge 1 not cooling,JA,Suncor - Mount Logan (49),Western (49-1),Union,NaN,NaN,NaN,NaN,NaN
3,3066355,115540631,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,Quality Auditing Compliance-LG,JA,Suncor - Firebag (48),Western (48-1),Union,NaN,NaN,NaN,NaN,NaN
4,3066355,115540628,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,Quality Auditing Compliance-LG,JA,Suncor - Firebag (48),Western (48-1),Union,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22050,3085206,116337518,2024-12-06,2024-06-13,FIXED,2,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,| Nights forgot to take a picture,CUL,Snow Lake (21),Eastern (21-1),Non-Union,20.0,NaN,NaN,NaN,NaN
22051,3042511,114552063,2024-04-05,2024-05-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,Quality Auditing Compliance-Photo required or ...,CUL,Snow Lake (21),NaN,Non-Union,20.0,NaN,NaN,NaN,NaN
22052,3038260,114381855,2024-04-30,2024-04-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,NaN,CUL,Snow Lake (21),NaN,Non-Union,20.0,NaN,NaN,NaN,NaN
22053,3164793,119637445,2024-01-09,NaT,OPN,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,Quality Auditing|Compliance-Photo required-LG,CUL,Snow Lake (21),Eastern (21-1),Non-Union,20.0,NaN,NaN,NaN,NaN


In [50]:
# Step 0: Strip spaces from all column names just in case
df.columns = df.columns.str.strip()
checkpoint_costs.columns = checkpoint_costs.columns.str.strip()
union_rates.columns = union_rates.columns.str.strip()
non_union_rates.columns = non_union_rates.columns.str.strip()
risk_ranking_weights.columns = risk_ranking_weights.columns.str.strip()

# Step 1: Merge checkpoint data (time + material)
df = df.merge(
    checkpoint_costs[['Checkpoint ID', 'Time taken to complete task (in mins)', 'Material Cost']],
    on='Checkpoint ID',
    how='left'
)

# Step 2: Add union status and combine rate tables
non_union_rates['Union / Non-Union'] = 'Non-Union'
union_rates['Union / Non-Union'] = 'Union'

combined_rates = pd.concat([non_union_rates, union_rates], ignore_index=True)

# Step 2.5: Rename to standard column name for ease
combined_rates = combined_rates.rename(columns={'Minute Rate': 'MinuteRate', 'Code': 'ResponsibleParty'})

# Step 3: Merge rate based on Responsible Party and Union / Non-Union
df = df.merge(
    combined_rates[['ResponsibleParty', 'MinuteRate', 'Union / Non-Union']],
    on=['ResponsibleParty', 'Union / Non-Union'],
    how='left'
)

# Step 4: Merge risk ranking weights
risk_ranking_weights = risk_ranking_weights.rename(columns={'Risk Ranking': 'RiskRanking', 'Weight': 'RiskWeight'})
df = df.merge(risk_ranking_weights, on='RiskRanking', how='left')

# Step 5: Use the newly merged columns
df['Time taken to complete task (in mins)'] = df['Time taken to complete task (in mins)_y']
df['Material Cost'] = df['Material Cost_y'].fillna(0)

# Step 6: Calculate re-do cost
df['Cost to re-do (based on hourly wage and multiplier)'] = (
    df['Time taken to complete task (in mins)'] *
    df['MinuteRate'] *
    df['Weights']
)

# Step 7: Total cost
df['Total cost to re-do task'] = (
    df['Cost to re-do (based on hourly wage and multiplier)'] +
    df['Material Cost']
)

In [52]:
df


,,DeficiencyID,DateCreated,DateFixed,CheckpointStatus,RiskRanking,Checklist ID,Checklist Category,Checklist Code,Checklist Item,...,Minute Rate (based on union/non-union),Cost to re-do (based on hourly wage and multiplier),Material Cost_x,Total cost to re-do task,Time taken to complete task (in mins)_y,Material Cost_y,MinuteRate,Weights,Time taken to complete task (in mins),Material Cost
0,3096177,116797739,2024-06-22,2024-06-07,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,NaN,0.000,NaN,0.000,10,NaN,0.407333,0.0,10,0.0
1,3229935,122354816,2024-10-26,2024-10-26,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,NaN,0.000,NaN,0.000,10,NaN,0.407333,0.0,10,0.0
2,3227124,122232426,2024-10-20,2024-10-20,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,NaN,0.000,NaN,0.000,10,NaN,0.407333,0.0,10,0.0
3,3066355,115540631,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,NaN,0.000,NaN,0.000,10,NaN,0.407333,0.0,10,0.0
4,3066355,115540628,2024-05-26,2024-05-27,FIXED,1,HN-2023-3004,Management/Administration,HN,Weekly Commissary,...,NaN,0.000,NaN,0.000,15,NaN,0.407333,0.0,15,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24111,3085206,116337518,2024-12-06,2024-06-13,FIXED,2,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,NaN,0.095,NaN,0.095,3,NaN,0.316667,0.1,3,0.0
24112,3042511,114552063,2024-04-05,2024-05-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,NaN,0.000,NaN,0.000,10,NaN,0.316667,0.0,10,0.0
24113,3038260,114381855,2024-04-30,2024-04-05,FIXED,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,NaN,0.000,NaN,0.000,10,NaN,0.316667,0.0,10,0.0
24114,3164793,119637445,2024-01-09,NaT,OPN,1,SLL-2023-1002,Culinary,SLL,Daily Culinary Report,...,NaN,0.000,NaN,0.000,10,NaN,0.316667,0.0,10,0.0


In [53]:
df.to_excel('Outputs.xlsx', index=False)